# Buenas Practicas en Python

>Tutorial hecho por Gonzalo y Claudio en EFX y algunas cosas modificadas/agregadas por mi.

## Algunas conclusiones valiosas

* Si los valores de una lista no se van a modificar, usar tuplas (esta conclusión es de otro tutorial)


* Usar diccionarios.


* Al trabajar con listas, siempre que se pueda, usar este orden de preferencia:
    1. map/lambda
    2. listas por comprensión (problema de ciclo for)
    3. **NO USAR**: ciclos for tradicionales, son los menos performantes.

OBS: El orden de 1. y 2. no está claro. A veces combiene usar uno y otras otro. MEDIR SIEMPRE.

* **No usar orientación a objetos** (clases, metodos, etc) por ser menos performante que map/lambda y diccionarios. 

---
* [Markdown tips](https://www.tutorialspoint.com/jupyter/jupyter_notebook_markdown_cells.htm)
* [Markdown tips2](https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed)
* [Markdown tips3](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)


# Table of contents
1. [Concatenar valores en python](#Concatenar)
2. [Importar otros archivos .py](#importar)
3. [Listas por comprension](#comprension)
4. [Map / Lambda](#map)
5. [NVL](#nvl)
6. [Variables globales vs diccionarios](#diccionarios)
7. [Listas](#listas)
  1. [Verificar si una lista tiene aunque sea un valor/posición](#verif)
  2. [Listas: Diferencia entre append y extend](#append)
  3. [Ordenamiento de listas](#ord)
    1. [Ejercicio de ordenamiento de listas](#ejord)
8. [Manejo de errores](#errores)

## Concatenar valores en python<a name="Concatenar"></a>

In [3]:
# Dada la siguiente lista
lista = ['1','2','3','0','-','J','R','D','C']

# Bien concatenada
c = ''.join(lista)

print (c)


1230-JRDC


In [6]:
# Mal concatenada
concatenada = ''
for s in lista:
    concatenada += s

print (concatenada)

1230-JRDC


## Importar otros archivos .py<a name="importar"></a>

Es más prolijo importar aquellas funciones que necesitamos anes que toda la librería

**Bien**

	import nombre_archivo_ejemplo as nombre
	nombre.funcion

	from nombre_archivo_ejemplo import funcion
	funcion

**Mal**

	from nombre_archivo_ejemplo import *


## Listas por comprension<a name="comprension"></a>

Para iterar y trabajar con todos los elementos de una lista, se pueden usar listas por comprensión.
El resultado de aplicar una lista por comprensión es otra lista, la cual tiene el resultado de aplicarle una función los elementos que cumplen con la condición explicitada.

Sintaxis:

**Lista_comprension = [resultado_a_devolver ciclo_for condición_if]**


#### Bien

In [11]:
lista = [1,2,3]

# Agregar valores sumando todos los valores de la lista
suma = sum([x for x in lista])
print("Resultado de la suma")
print(suma)

# Filtrar valores, solo dejar los mayores a 1

filt = [x for x in lista if x > 1]
print("Resultado del filtrado")
print(filt)


Resultado de la suma
6
Resultado del filtrado
[2, 3]


#### Mal - no hacer esto (es menos performante ??)

In [17]:
aux = 0

for n in lista:
    if n > 1:
        aux += n
        
print (aux)

5


La desventaja es que igualmente estamos usando un ciclo for para recorrer cada elemento de la lista, si bien es más performante usar listas por comprensión (ya que Python las trabaja de forma más óptima que con ciclos for y la función in()), no evitamos tener que usar un ciclo.

## Map / Lambda<a name="map"></a>

La función map recorre y aplica una función determinada sobre un conjunto de valores (Lista) y retorna una lista. 

**No se le puede pasar por parámetro ninguna variable a la función que invocamos desde la función map.** Si queremos realizar esto, se puede crear un diccionario en el scope global y luego, dentro de la función que llamamos desde la función map, hacer uso de este diccionario según corresponda.

Sintaxis:
    
    map(funcion, lista)   # funcion debe definirse antes
    
    map(lambda x:< >, lista)

Según Claudio y Gonzalo, usar map/lambda es más performante que usar listas por comprensión (yo no lo verifique).

a) *Listas por comprensión:*

**Saldo_cuotas_eq_0 = len([x for x in variables_tarjetas['saldo_cuotas_ult'] if x == 0 ])**

b) *Map y lambda:*

**Saldo_cuotas_eq_0 = sum(map(lambda x: 1 if x == 0 else 0,variables_tarjetas['saldo_cuotas_ult']))**

Si bien ambos códigos devolverán lo mismo, la diferencia reside en que no estamos usando un ciclo for.
En una línea no vamos a notar mucha mejora de tiempos, pero si en el acumulado, en el código completo.

In [26]:
#suma = sum([x for x in lista])
def func(x):
    return x

# Con map se recorre toda la lista - la función devuelve el valor o puede realizar alguna operación.
sum(map(func,lista))

6

In [30]:
# Ahora la función se hace usando lambda
sum(map(lambda x:x ,lista))

6

In [34]:
# Esto da error
list(map(lambda x,y:x + y ,lista))

TypeError: <lambda>() missing 1 required positional argument: 'y'

In [36]:
# Para hacer lo anterior hay que usar reduce asi:
import functools as ft                 #Reduce esta en esta libreria

ft.reduce(lambda x,y:x + y ,lista)     # Se reemplaza 'map' por 'reduce' !!

6

## NVL<a name="nvl"></a> 

### Operaciones con la instancia de objeto None

No se puede realizar ninguna operación matemática con alguna etiqueta que apunte a una instancia del objeto None.
En otras palabras, no se pueden realizar las siguientes operaciones:

A = None
A*2
A/2

Tampoco se puede concatenar None

print("texto:" + None) -> da error

Habría que hacer esto:
print("texto:" str(None))


In [48]:
# Se puede hacer una función propia NVL
#--------NVL-------------#
def NVL(var, val):
    if var is None:
        return val
    else:
        return var

#### Pero Python ya tiene implementado un NVL ...

In [49]:
# En estos casos, usará el valor de 'x' si no es None o False, sino será '0'
x = 1
x = None
x = False

print (x or 0)

0


## Variables globales vs diccionarios<a name="diccionarios"></a>

Al trabajar con variables globales, notamos que no siempre estaban llegando a todas las funciones, es decir, no siempre dentro del scope de una función lográbamos operar con el valor de alguna variable definida como global (global nombre_variable).

En vez de eso, se usaron diccionarios, definidos dentro del scope global.
De esta forma pudimos realizar un pasamano de variables transparente entre diferentes funciones a lo largo de todo el script en Python.

El manejo de diccionarios nos resultó bastante performante a la hora de trabajar con estructuras que acumulen otras estructuras de datos (listas, tuplas, diccionarios o simples variables integer, float, string) categorizados bajo una misma fuente de datos.

Además al parecer, *Python es poco performante  al momento de tener que mantener el seguimiento de las referencias a variables globales.*


### Diccionarios: concatenación

In [51]:
# Dados estos diccionarios
dict_1 = {'dict_1_clave1':[],'dict_1_clave2':[],'dict_1_clave3':[]}
dict_2 = {'dict_2_clave1':[],'dict_2_clave2':[],'dict_2_clave3':[]}
dict_3 = {'dict_3_clave1':[],'dict_3_clave2':[],'dict_3_clave3':[]}
dict_4 = {'dict_4_clave1':[],'dict_4_clave2':[],'dict_4_clave3':[]}
dict_5 = {'dict_5_clave1':[],'dict_5_clave2':[],'dict_5_clave3':[]}

# Los queremos fusionar en el siguiente diccionario
dict_fusion = {}

dict_fusion = dict(dict_fusion, ** dict_1)
dict_fusion

{'dict_1_clave1': [], 'dict_1_clave2': [], 'dict_1_clave3': []}

In [55]:
# fusionando con los otros:
dict_fusion = dict(dict_fusion, ** dict_2)
dict_fusion = dict(dict_fusion, ** dict_3)
dict_fusion = dict(dict_fusion, ** dict_4)
dict_fusion = dict(dict_fusion, ** dict_5)

dict_fusion

{'dict_1_clave1': [],
 'dict_1_clave2': [],
 'dict_1_clave3': [],
 'dict_2_clave1': [],
 'dict_2_clave2': [],
 'dict_2_clave3': [],
 'dict_3_clave1': [],
 'dict_3_clave2': [],
 'dict_3_clave3': [],
 'dict_4_clave1': [],
 'dict_4_clave2': [],
 'dict_4_clave3': [],
 'dict_5_clave1': [],
 'dict_5_clave2': [],
 'dict_5_clave3': []}

## Listas <a name="listas"></a>

### Verificar si una lista tiene aunque sea un valor/posición<a name="verif"></a>

__Bien__:

    If lista:

__Mal 1__

    If len(lista) > 0:

__Mal 2__
		 
    If lista != []:
    
    
En caso de necesitar saber si la lista tiene más de dos valores/posiciones, usar función len(), sino alcanza con preguntar por la lista en la condición.

Para verificar por valores True o False

__Bien__
		
        If x:

__Mal__

        If x == True:


### Listas: Diferencia entre append y extend<a name="append"></a> 
-**append** crea una nueva posición en la lista y agrega el objeto pasado por parámetro.

-**extend** crea una nueva posición en la lista si el elemento pasado por parametro es un objeto no iterable (integer, float, string, etc), si el elemento es un objeto iterable (lista), agrega cada elemento del objeto iterable pasado por parámetro en una nueva posición. Es decir, **cambia el tipo de dato agregado de lista a objetos escalares**


In [57]:
#Ejemplo 1 - append
lista_1 = [1,2,3,4,5]
lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]

lista_3.append(lista_1)

lista_3


['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 [1, 2, 3, 4, 5]]

In [59]:
# Ejemplo 2 - extend
lista_1 = [1,2,3,4,5]
lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]

lista_3.extend(lista_1)
lista_3

['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 1,
 2,
 3,
 4,
 5]

In [62]:
#Agregamos un diccionario a la lista usando 'extend'
lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]
dict_1 = {'dict_1_clave1':[],'dict_1_clave2':[],'dict_1_clave3':[]}

lista_3.extend(dict_1)
lista_3

['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 'dict_1_clave1',
 'dict_1_clave2',
 'dict_1_clave3']

In [63]:
#Agregamos un diccionario a la lista usando 'append'
lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]
dict_1 = {'dict_1_clave1':[],'dict_1_clave2':[],'dict_1_clave3':[]}

lista_3.append(dict_1)
lista_3

['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 {'dict_1_clave1': [], 'dict_1_clave2': [], 'dict_1_clave3': []}]

In [65]:
# Agregando una tupla a una lista - extend
tupla_1 = (1,2,3,4,5,6,7)

lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]
lista_3.extend(tupla_1)

lista_3

['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 1,
 2,
 3,
 4,
 5,
 6,
 7]

In [66]:
# Agregando una tupla a una lista - append
tupla_1 = (1,2,3,4,5,6,7)

lista_3 = ['soy un string en la posicion 1',['soy una lista en la posicion 2']]
lista_3.append(tupla_1)

lista_3

['soy un string en la posicion 1',
 ['soy una lista en la posicion 2'],
 (1, 2, 3, 4, 5, 6, 7)]

### Ordenamiento de listas<a name="ord"></a>

<blockquote>
Existe la función sorted que devuelve el objeto iterable pasado por parámetro ordenado según algún criterio.
<br>

Sintaxis:<br>
**sorted(iterable[, key][, reverse])**
<br>

Donde:<br>
iterable - sequence (string, tuple, list) or collection (set, dictionary, frozen set) or any iterator 
reverse (Optional) - <br>If true, the sorted list is reversed (or sorted in Descending order)
key (Optional) - <br>function that serves as a key for the sort comparison
</blockquote>

In [68]:
# ordenando una lista
lista_1 = [4,5,23,2,1]
lista_1 = sorted(lista_1)

lista_1

[1, 2, 4, 5, 23]

In [70]:
#Ordenando una lista de listas
# Como se ve, solo ordena la lista y no los elementos que contienen las listas en su interior.
lista_1 = [[64,13,43],[6,3,5]]
lista_1 = sorted(lista_1)

lista_1

[[6, 3, 5], [64, 13, 43]]

In [35]:
#Ordenando una lista de tuplas, usando la clausula key:
def segundo(elem):
    return elem[1]

lista_1 = [(2, 2), (3, 4), (4, 1), (1, 3)]
lista_1 = sorted(lista_1, key=segundo)
#lista_1 = sorted(lista_1, key=lista_1[1])  # <- esto da error de tipo (TypeError)

lista_1

[(4, 1), (2, 2), (1, 3), (3, 4)]

#### Ejercicio de ordenamiento de listas<a name="ejord"></a>

Dados 3 diccionarios, ordenarlos por:

* menor valor de pos
* mayor valor de aportado_cant
* mayor valor de aportado_fh_num
* menor valor de cod_postal

##### Todos los diccionarios a una lista

In [58]:
#dict_1={'aportado_fh':'2012-10','aportado_fh_num':'20180101','aportado_cant':4,'cod_postal':1406,'mes':360,'cpa':'DDCR10','geo_nse':'NA','provincia':'P','aglomerado':'C','avg_tc_limite_credito':1234.123,'pos':1,'cpa_num':331,'impacto_codigo_num':12,'tasa_mora':23123.12321,'tasa_p_mora':12312.4312}
#dict_2={'aportado_fh':'2017-10','aportado_fh_num':'20180102','aportado_cant':2,'cod_postal':1407,'mes':180,'cpa':'DDCR12','geo_nse':'NA','provincia':'P','aglomerado':'C','avg_tc_limite_credito':1234.123,'pos':2,'cpa_num':331,'impacto_codigo_num':12,'tasa_mora':23123.12321,'tasa_p_mora':12312.4312}
#dict_3={'aportado_fh':'2017-09','aportado_fh_num':'20171201','aportado_cant':5,'cod_postal':1400,'mes':60,'cpa':'DDCR33','geo_nse':'NA','provincia':'P','aglomerado':'C','avg_tc_limite_credito':1234.123,'pos':3,'cpa_num':331,'impacto_codigo_num':12,'tasa_mora':23123.12321,'tasa_p_mora':12312.4312}
# diccionarios modificados por simplicidad
dict_1={'Nombre':'PRIMERO','aportado_fh_num':'20180101','aportado_cant':1,'cod_postal':1401,'pos':1}
dict_2={'Nombre':'SEGUNDO','aportado_fh_num':'20180102','aportado_cant':2,'cod_postal':1402,'pos':2}
dict_3={'Nombre':'TERCERO','aportado_fh_num':'20180103','aportado_cant':3,'cod_postal':1403,'pos':1}


# Juntamos todos los diccionarios en una lista
direcciones_completas = []

# Como queremos mantener la estrutura de los diccionarios usamos append
direcciones_completas.append(dict_1)
direcciones_completas.append(dict_2)
direcciones_completas.append(dict_3)

direcciones_completas     # lista de diccionarios

[{'Nombre': 'PRIMERO',
  'aportado_fh_num': '20180101',
  'aportado_cant': 1,
  'cod_postal': 1401,
  'pos': 1},
 {'Nombre': 'SEGUNDO',
  'aportado_fh_num': '20180102',
  'aportado_cant': 2,
  'cod_postal': 1402,
  'pos': 2},
 {'Nombre': 'TERCERO',
  'aportado_fh_num': '20180103',
  'aportado_cant': 3,
  'cod_postal': 1403,
  'pos': 1}]

##### Ordenar lista 'direcciones_completas' por 'pos'

In [59]:
#Ordenar por menor valor de 'pos'
def pos(lista):
    return lista['pos']

lista_ordenada1 = sorted(direcciones_completas, key=pos, reverse=False)

#direcciones_completas[1]['pos']
lista_ordenada1

[{'Nombre': 'PRIMERO',
  'aportado_fh_num': '20180101',
  'aportado_cant': 1,
  'cod_postal': 1401,
  'pos': 1},
 {'Nombre': 'TERCERO',
  'aportado_fh_num': '20180103',
  'aportado_cant': 3,
  'cod_postal': 1403,
  'pos': 1},
 {'Nombre': 'SEGUNDO',
  'aportado_fh_num': '20180102',
  'aportado_cant': 2,
  'cod_postal': 1402,
  'pos': 2}]

In [60]:
#mayor valor de 'aportado_cant'
def aportado_cant(lista):
    return lista['aportado_cant']

lista_ordenada2 = sorted(lista_ordenada1, key=aportado_cant, reverse=True)
lista_ordenada2

# Si sigo así tendo que hacer una función por cada ordenamiento - esto se puede evitar usando lambda (ver abajo)

[{'Nombre': 'TERCERO',
  'aportado_fh_num': '20180103',
  'aportado_cant': 3,
  'cod_postal': 1403,
  'pos': 1},
 {'Nombre': 'SEGUNDO',
  'aportado_fh_num': '20180102',
  'aportado_cant': 2,
  'cod_postal': 1402,
  'pos': 2},
 {'Nombre': 'PRIMERO',
  'aportado_fh_num': '20180101',
  'aportado_cant': 1,
  'cod_postal': 1401,
  'pos': 1}]

#### Solucion de Claudio y Gonzalo

Como dijimos, si vamos a usar reverse = True (De mayor a menor), aquellos campos que necesitemos el menor tendremos que negarlos, y viceversa si no ponemos la cláusula reverse.

Tener en cuenta que, al ordenar por una tupla, la función sorted ira desde la primer posición hasta la última, es decir, si el valor de la primer posición de la tupla es igual a todas las demás, ordenara por el segundo campo y así hasta el final de los campos. Entonces, **si hay algún campo que sea más relevante que otro se debería meter al principio de la tupla.**

Nótese que a los campos pos y cod_postal, se les agrego un “-“  delante para negarlos y asi ordenar con un criterio opuesto al resto de los campos.

In [61]:
# Solucion de Claudio y Gonzalo
lista_ordenada = sorted(direcciones_completas, 
        key = lambda dir: (-dir['pos'],dir['aportado_cant'],dir['aportado_fh_num'],-dir['cod_postal']),
        reverse=True)

lista_ordenada

[{'Nombre': 'TERCERO',
  'aportado_fh_num': '20180103',
  'aportado_cant': 3,
  'cod_postal': 1403,
  'pos': 1},
 {'Nombre': 'PRIMERO',
  'aportado_fh_num': '20180101',
  'aportado_cant': 1,
  'cod_postal': 1401,
  'pos': 1},
 {'Nombre': 'SEGUNDO',
  'aportado_fh_num': '20180102',
  'aportado_cant': 2,
  'cod_postal': 1402,
  'pos': 2}]

In [62]:
#Obteniendo el primero valor de la lista y accediento por su clave a cod_postal:
print (lista_ordenada[0]['cod_postal'])

1403


## Manejo de errores<a name="errores"></a>
Fuente: [Python Tutorial: Using Try/Except Blocks for Error Handling](https://www.youtube.com/watch?v=NIWwJbo-9_8)

Para evitar el trace completo en python se pueden usar las sentencias _try & Except_

In [13]:
!echo "hola que tal" > test.txt;ls

Python-BuenasPracticas.html     buenas practicas pythonEFX.docx
Python-BuenasPracticas.ipynb    test.txt


In [6]:
# Generamos un error 
f = open("archivo_incorrecto")

FileNotFoundError: [Errno 2] No such file or directory: 'archivo_incorrecto'

In [8]:
# Capturando el error 'FileNotFoundError' ...
try:
    f = open("archivo_incorrecto")
    a = b
except FileNotFoundError:
    print("No se ha encontrado el archivo")
except Exception as e:
    print(f"Otro error:{e}")

No se ha encontrado el archivo


In [11]:
# Si el archivo es correcto pero hay otro error, se puede capturar poniendo ULTIMO el error mas general
try:
    f = open("test.txt")
    a = b
except FileNotFoundError:
    print("No se ha encontrado el archivo")
except Exception as e:
    print(f"Otro error:'{e}'")

Otro error:'name 'b' is not defined'


In [21]:
# Usando 'else'
try:
    f = open("test.txt")
except FileNotFoundError:
    print("No se ha encontrado el archivo")
except Exception as e:
    print(f"Otro error:'{e}'")
else:
    print("Esto corre si NO hay una 'Exception'")
    print("Es para manejar los errores con mas precision\n")
    msg = f.read()
    print('#'*len(msg) + f"\n{msg}" + '#'*len(msg))
    f.close()

Esto corre si NO hay una 'Exception'
Es para manejar los errores con mas precision

#############
hola que tal
#############


In [28]:
# Usando finally
try:
    f = open("test.txt")
    a = b
except FileNotFoundError:
    print("No se ha encontrado el archivo")
except Exception as e:
    print(f"Otro error:'{e}'")
else:
    print("Esto corre si NO hay una 'Exception'")
    print("Es para manejar los errores con mas precision\n")
    msg = f.read()
    print('#'*len(msg) + f"\n{msg}" + '#'*len(msg))
    f.close()
finally:
    print("\n-------------------------------------------------")
    print("Esto corre SIEMPRE, si hay una 'Exception' o no")
    print("-------------------------------------------------")

Otro error:'name 'b' is not defined'

-------------------------------------------------
Esto corre SIEMPRE, si hay una 'Exception' o no
-------------------------------------------------


In [30]:
# Excepciones manuales con 'raise'
try:
    f = open("test.txt")
    if f.name == "test.txt":
        raise Exception(f"Este archivo no me gusta:{f.name}")
except FileNotFoundError:
    print("No se ha encontrado el archivo")
except Exception as e:
    print(f"Error!:'{e}'")
else:
    print("Esto corre si NO hay una 'Exception'")
finally:
    print("finally ...")

Error!:'Este archivo no me gusta:test.txt'
finally ...
